In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Crie uma frase sobre: {assunto}")

In [3]:
chain = prompt | model

In [4]:
chain

ChatPromptTemplate(input_variables=['assunto'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assunto'], template='Crie uma frase sobre: {assunto}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021A62595C10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021A62667650>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='')

In [5]:
chain.invoke({"assunto": "Python"})

AIMessage(content='Python é uma linguagem de programação poderosa e versátil, capaz de transformar ideias em soluções de forma simples e eficiente.', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 14, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bdc477a2-7e01-4c99-a574-d49a746f06ae-0')

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [7]:
chain.invoke({"assunto": "Python"})

'Python é uma linguagem de programação versátil e poderosa, que permite desenvolver soluções inovadoras e eficientes em diversas áreas.'

In [8]:
input = {"assunto":"Python"}

In [9]:
prompt_format = prompt.invoke(input)
prompt_format

ChatPromptValue(messages=[HumanMessage(content='Crie uma frase sobre: Python')])

In [10]:
resposta = model.invoke(prompt_format)
resposta

AIMessage(content='"Python é uma linguagem de programação versátil e poderosa que facilita a vida dos desenvolvedores em diversas áreas de atuação."', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 14, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b7329e37-6de6-45b2-a367-237cf1eb1672-0')

In [11]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto {assunto}")

output_parser = StrOutputParser()

In [12]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)

chain.invoke({"assunto": "Python"})

d:\Udemy\2025\agents\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'assunto': 'Python',
 'text': 'Python é uma linguagem de programação versátil e poderosa, perfeita para desenvolver projetos de qualquer tamanho e complexidade.'}

In [13]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
caminhos = [
    "files/LLM.pdf",
    ]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [16]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [17]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model
)
retriever = vectorstore.as_retriever(search_type='mmr')

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()

template_str = '''Responda as perguntas do usuário com base no contexto fornecido.

Contexto: {contexto}

Pergunta: {pergunta}'''

template = ChatPromptTemplate.from_template(template_str)
output_parser = StrOutputParser()

In [19]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_retrieval = RunnableParallel(
    {"pergunta": RunnablePassthrough(),
     "contexto": retriever}
)

chain = setup_retrieval | template | model | output_parser

In [20]:
chain.invoke("O que é LLM?")

'LLM significa Large Language Models, que são modelos de linguagem de grande escala. Eles são utilizados para desenvolver aplicativos prontos para produção e analisar textos, podendo ajudar a analisar emoções e opiniões, entre outras tarefas.'

In [21]:
retriever.invoke('O que é o LLM?')

[Document(page_content='. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação', metadata={'source': 'files/LLM.pdf', 'page': 7}),
 Document(page_content='.0, LLaMA, Alpaca e Vicuna. O GPT-4 também é lançado, estabelecendo um novo referencial tanto em tamanho de parâmetros quanto em desempenho.', metadata={'source': 'files/LLM.pdf', 'page': 2}),
 Document(page_content='E-BOOK \nUm guia compacto sobre Large Language Models (LLM)', metadata={'source': 'files/LLM.pdf', 'page': 0}),
 Document(page_content='.   ANÁLISE DE SENTIMENTO  Frequentemente, uma tarefa difícil de quantificar, os LLMs podem ajudar a analisar emoções e opiniões a partir de um texto. Isso pode ajudar as organizações a coletarem os dados e o feedback necessários para melhorar a satis